# Siamese Training: Training the Model

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### General Steps to Follow

1. Importing Packages
2. Defining x_train, x_test, y_train, y_test
3. Building and training the siamese network
4. Model Evaluation

## 1) Importing Packages

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D, MaxPooling2D, Layer,Concatenate, Lambda, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

### ----------------------------------------------------------------------------------------------------------------------------------------------------------

## 2) Defining x_train, x_test, y_train, y_test

#### Loading the training and test data from "other data" folder

In [6]:
train_data = np.load("/content/drive/MyDrive/OpenCV 2023/train_data.npy", allow_pickle = True)
test_data = np.load("/content/drive/MyDrive/OpenCV 2023/test_data.npy"  , allow_pickle = True)

* x_train and x_test will contain pairs of the anchor image and the validation image(positive or negative image).
* y_train and y_test will contain the label of each pair:
  - 1 if the pairs are similar images.
  - 0 if the pairs are different images.

In [7]:
x_train = train_data[:,0:2]
y_train = train_data[:,2]
x_test = test_data[:,0:2]
y_test = test_data[:,2]

#### Reshaping the input

In [8]:
x1_train = x_train[:,0]                   #anchor images
x1_train = np.array(x1_train.tolist())
x2_train = x_train[:,1]                   #validation images(positive/negative)
x2_train = np.array(x2_train.tolist())

x1_test = x_test[:,0]                    #anchor images
x1_test = np.array(x1_test.tolist())
x2_test = x_test[:,1]                    #validation images(positive/negative)
x2_test = np.array(x2_test.tolist())

y_train = tf.convert_to_tensor(y_train.tolist())
y_test = tf.convert_to_tensor(y_test.tolist())

#### Checking the shapes

In [9]:
print("Train Data:")
print("Shape of anchor images    : ", x1_train.shape)
print("Shape of validation images: ", x2_train.shape)
print("Shape of labels           : ", y_train.shape)

print("--------------------------------------------------")

print("Test Data:")
print("Shape of anchor images    : ", x1_test.shape)
print("Shape of validation images: ", x2_test.shape)
print("Shape of labels           : ", y_test.shape)

Train Data:
Shape of anchor images    :  (961, 105, 105, 3)
Shape of validation images:  (961, 105, 105, 3)
Shape of labels           :  (961,)
--------------------------------------------------
Test Data:
Shape of anchor images    :  (240, 105, 105, 3)
Shape of validation images:  (240, 105, 105, 3)
Shape of labels           :  (240,)


### ----------------------------------------------------------------------------------------------------------------------------------------------------------

## 3) Building and training the siamese network

### 3.1 Building the base of the network

In [10]:
inp_shape = [105,105,3]

In [68]:
def make_base_network():
    model = Sequential(
        [
            Input(shape = inp_shape, name = "input_image"),

            Conv2D(32, (3, 3), activation = 'relu'),
            MaxPooling2D(64,(2,2), padding = 'same'),

            Conv2D(64, (3, 3), activation = 'relu'),
            MaxPooling2D(64,(2,2), padding = 'same'),

            Flatten(),

            Dense(1024, activation = 'relu'),
            BatchNormalization(),
        ], name = "BaseNetwork"
    )

    return model

In [69]:
base_model = make_base_network()

In [39]:
base_model.summary()

Model: "BaseNetwork"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 103, 103, 32)      896       
                                                                 
 max_pooling2d_20 (MaxPooli  (None, 52, 52, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_21 (Conv2D)          (None, 50, 50, 64)        18496     
                                                                 
 max_pooling2d_21 (MaxPooli  (None, 25, 25, 64)        0         
 ng2D)                                                           
                                                                 
 flatten_6 (Flatten)         (None, 40000)             0         
                                                                 
 dense_9 (Dense)             (None, 128)               

### --------------------------------------------------------------------------------

### 3.2 Building tthe L1Dist layer

In [14]:
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super(L1Dist, self).__init__(**kwargs)

    def call(self, anchor, validation):
        return tf.abs(anchor - validation)

### --------------------------------------------------------------------------------

### 3.3 Defining the siamese model

In [15]:
def make_siamese_model():

    # Anchor input image to the network
    anc_image = Input(shape = inp_shape, name = "input_image")

    # Validation input image to the network
    validation_image = Input(shape = inp_shape, name = "Validation_image")

    # creating a base model
    base_model = make_base_network()

    # Encoding the anchor image
    anchor = base_model(anc_image)

    # Encoding the validation image
    validation = base_model(validation_image)

    # Using L1Dist Layer to calculate the L1 distance between the two encodings
    distance_layer = L1Dist()
    distance_layer._name = "distance_layer"
    distance = distance_layer(anchor, validation)


    # Defining the output layer
    output_layer = Dense(1, activation = 'linear')(distance)

    siamese_model = Model(inputs = [anc_image, validation_image], outputs = output_layer, name = "SiameseNetwork")

    return siamese_model

In [100]:
siamese_model = make_siamese_model()
siamese_model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_image (InputLayer)    [(None, 105, 105, 3)]        0         []                            
                                                                                                  
 Validation_image (InputLay  [(None, 105, 105, 3)]        0         []                            
 er)                                                                                              
                                                                                                  
 BaseNetwork (Sequential)    (None, 1024)                 4098451   ['input_image[0][0]',         
                                                          2          'Validation_image[0][0]']    
                                                                                     

### --------------------------------------------------------------------------------

### 3.4 Compiling and training the siamese model

In [101]:
siamese_model.compile(
    optimizer = Adam(learning_rate = 0.01),
    loss = BinaryCrossentropy(from_logits = True)
)

In [124]:
siamese_model.fit([x1_train, x2_train], y_train, epochs = 10, batch_size = 64)

Epoch 1/10
16/16 [==============================] - 5s 291ms/step - loss: 0.2155
Epoch 2/10
16/16 [==============================] - 5s 298ms/step - loss: 0.1977
Epoch 3/10
16/16 [==============================] - 5s 294ms/step - loss: 0.2451
Epoch 4/10
16/16 [==============================] - 5s 295ms/step - loss: 0.2140
Epoch 5/10
16/16 [==============================] - 5s 300ms/step - loss: 0.1746
Epoch 6/10
16/16 [==============================] - 5s 298ms/step - loss: 0.1925
Epoch 7/10
16/16 [==============================] - 5s 295ms/step - loss: 0.1774
Epoch 8/10
16/16 [==============================] - 5s 296ms/step - loss: 0.1897
Epoch 9/10
16/16 [==============================] - 5s 300ms/step - loss: 0.1727
Epoch 10/10
16/16 [==============================] - 5s 293ms/step - loss: 0.1631


### ----------------------------------------------------------------------------------------------------------------------------------------------------------

## 4) Model Evaluation

In [74]:
def model_eval(y, y_hat):
    m = y.shape[0]

    y_hat = y_hat.numpy()
    y_temp = np.zeros(m, dtype = "int")
    for i in range(len(y_hat)):
        if(y_hat[i] >= 0.5):
            y_temp[i] = 1
        else:
            y_temp[i] = 0


    accuracy = 100*(np.sum(y == y_temp)/m)
    print("Accuracy =", accuracy)

#### Evaluation on training data

In [136]:
output1 = siamese_model.predict([x1_train, x2_train])

31/31 [==============================] - 2s 78ms/step


In [137]:
y_hat = tf.nn.sigmoid(output1)
y = y_train
model_eval(y, y_hat)

Accuracy = 91.88345473465141


#### Evaluation on test data

In [138]:
output2 = siamese_model.predict([x1_test, x2_test])

8/8 [==============================] - 1s 73ms/step


In [139]:
y_hat = tf.nn.sigmoid(output2)
y = y_test
model_eval(y, y_hat)

Accuracy = 89.16666666666667


### Saving the model

In [135]:
siamese_model.save("/content/drive/MyDrive/OpenCV 2023/siamese_model.h5")
#3: 91.88, 89.16

### Loading the model

In [130]:
siamese_model = tf.keras.models.load_model('/content/drive/MyDrive/OpenCV 2023/siamese_model3.h5',
                                   custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

In [ ]:
import tensorflow as tf

In [ ]:
tf. __version__

'2.14.0'

In [56]:
model.history